# Description/notes
This notebook is named interpolation because it makes it easy to remember what it does, that is to complete null values.
But it does so without performing mathematical interpolation. Instead it looks at all the information given to find the best option.

In [1]:
import datetime
import pandas as pd
import numpy as np

# Function

In [2]:
def interpolation_from_other_rows(key_series,
                                  value_series,
                                  disambiguate = True, 
                                  correct_potential_mistakes = False,
                                  threshold_abs = 4,
                                  threshold_pct = 0.7):
    """Completes null values in a series (value_series) by looking at all the rows from another series (key_series).
    
    If unique associations can be found e.g.: Paris -> France they are directly used to complete null values.
    
    In case of ambiguities and if the disambiguate parameter is True the most frequent value can be found using
    threshold_abs (how many times a combination key-value must appear to be used) and threshold_pct (frequency in
    relation to other combinations).
    
    In this case it is also possible to "correct" values not equal to the most frequent value found using threshold_abs
    and threshold_pct. For this, set the correct_potential_mistakes parameter to True.
    
    Args:
        key_series: pd.Series (the function has been tested only with string Series - dtype object)
        value_series: pd.Series (the function has been tested only with string Series - dtype object)
        disambiguate: bool (True or False)
        correct_potential_mistakes: bool (True or False)
        threshold_abs: integer
        threshold_pct: float
    
    
    Warning:
        This function is meant for key_series and value_series which have unique or "unique enough" combinations.
        It is recommanded to do use it in combination with other methods for completing missing values (e.g. using
        databases like Geonames) and do a thorough verification of the results.
        Some examples could be:
            - first name -> gender 
            - region -> country
            - client -> person responsible for a client
            - company name -> company type
    
    Returns:
        np.array of completed values
        
    
    """
    
    # Verifies arguments validity 
    if threshold_abs < 1:
        raise ValueError('Minimal accepted value for threshold_abs is 1')
    
    if not isinstance(threshold_abs,int):
        raise TypeError('threshold_abs should be of type int')
    
    if threshold_pct <= 0.5:
        raise ValueError('Minimal accepted value for threshold_pct is strictly above 0.5')
        
    
    # Creates a DataFrame from key and series and a copy of this df for later
    df_key_value = pd.DataFrame({'Key':key_series,'Value':value_series})
    df_copy = df_key_value.copy(deep = True)

    # Drops null lines and reset index
    df_key_value.dropna(axis = 0,how = 'any',inplace = True)
    df_key_value.reset_index(drop = True,inplace = True)

    # Tuple-izes key and values series
    combinations = pd.Series(df_key_value.itertuples(index = False))

    # Transforms tuple-ized key and values series to a DataFrame in order to join it to the previously created df
    combinations.name = 'Combination'
    df_combinations = pd.DataFrame(combinations)
    df_combinations = df_combinations.join(df_key_value)

    # Counts unique combinations per key then maps it to a column   
    unique_comb_per_key = df_combinations.groupby('Key')['Combination'].nunique()
    df_combinations['Count unique Combination per key'] = df_combinations['Key'].map(unique_comb_per_key)

    # Counts the frequency of the combinations then maps it to a column
    count_comb_total = df_combinations['Combination'].value_counts()
    df_combinations['Count Combination'] = df_combinations['Combination'].map(count_comb_total)
    df_combinations = df_combinations.drop_duplicates()

    # Counts the number of all combinations for each key
    count_all_comb_per_key = df_combinations.groupby(['Key'])['Count Combination'].sum(axis = 0)
    df_combinations['Count all combinations per key'] = df_combinations['Key'].map(count_all_comb_per_key)

    # Divides to find percentage
    df_combinations['Count Combination (Pct)'] = df_combinations['Count Combination']/df_combinations['Count all combinations per key']

    # Gets all mappings that fit requirements
    ## (Above percentage threshold (default more than 70% of the values) 
    ## AND above absolute threshold (default more than 4 occurences)) 
    ## OR (Count unique == 1 ) which means there is only one unique association such as Paris -> France.
    if disambiguate == True:
        df_combinations = df_combinations[((df_combinations['Count Combination (Pct)'] >= threshold_pct) & 
                                          (df_combinations['Count Combination'] >= threshold_abs)) | 
                                          (df_combinations['Count unique Combination per key'] == 1)]

    else:
        df_combinations = df_combinations[df_combinations['Count unique Combination per key'] == 1]


    # Merges the new values
    df_combinations = df_combinations.rename(columns = {'Value':'New values'})
    df_copy = df_copy.merge(df_combinations[['Key','New values']],on = ['Key'], how = 'left')

    # Depending on the arguments configuration some values can get lost in the way, we retrieve them here
    df_copy.loc[(df_copy['Value'].notna()) &
                (df_copy['New values'].isna()),'New values'] = df_copy['Value']

    # What to do if the most frequent value is not in accordance with some rows
    ## Let the values as they are
    if correct_potential_mistakes == False:
        # If old != new and both aren't null take the old value back
        df_copy.loc[(df_copy['New values'] != df_copy['Value']) & 
                    (pd.notna(df_copy[['Value','New values']]).all(axis = 1)),'New values'] = df_copy['Value']

    ## Else no need to do anything the new values can be returned as they are

    return df_copy['New values'].values

# Usage
Uncomment the last line to see the example.

In [3]:
class interpolation_example(): # class used only for the namespace
    
    # creating data
    dates = [datetime.date(2018,1,1)+datetime.timedelta(i) for i in range(50)]
    items_id = np.arange(1,51)
    cities = ['Firenze','London','London','Marseille','Marseille']
    cities += ['Paris' for i in range(21)]
    cities += ['Firenze' for i in range(24)]
    
    countries = ['Deutschland','Großbritannien',np.nan,'Frankreich','Frankreich']
    countries += np.random.choice(['Frankreich','Großbritannien',np.nan],21, p = [0.8,0.1,0.1]).tolist()
    countries += np.random.choice(['Italien',np.nan],24, p = [0.4,0.6]).tolist()

    data = {'Purchase_date': dates,'Item_ID':items_id,'City':cities,'Country':countries}
    
    # Creating the DataFrame
    df = pd.DataFrame(data)
    
    # Replacing "nan" string with true nulls (this happened because numpy casted np.nan to as tring with np.random.choice)
    df.replace('nan',np.nan,inplace = True)

    case1 = 'Country guessed if unequivocal'
    case2 = 'Country guessed with statistics'
    case3 = 'Country guessed with statistics + correction'



    # case1 : one unique associated country found for a city -> complete null values by this country
    df[case1] = interpolation_from_other_rows(key_series = df['City'],
                                              value_series = df['Country'],
                                              disambiguate = False)

    # case2 : more than one associated country found for the same city -> find most frequent value
    df[case2] = interpolation_from_other_rows(key_series = df['City'],
                                              value_series = df['Country'],
                                              disambiguate = True,
                                              correct_potential_mistakes = False)

    # case3 : more than one associated country found for the same city -> find most frequent value and "correct" values
    # not equal to the most frequent value
    df[case3] = interpolation_from_other_rows(key_series = df['City'],
                                              value_series = df['Country'],
                                              disambiguate = True,
                                              correct_potential_mistakes = True)
    results = df
    
#interpolation_example.results